In [18]:
import numpy as np
import pandas as pd
import plotly

from plotly.subplots import make_subplots

plotly.__version__

plotly.offline.init_notebook_mode(connected=True)

'4.0.0'

***See Returns.ipynb for an example of using plotly with pandas Series and DataFrames***

***Reading https://plot.ly/python/creating-and-updating-figures/ is worthwhile***

## Simple example

In [12]:
import plotly
import plotly.graph_objs as go


trace0 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = [trace0, trace1]

plotly.offline.iplot(data)

## Wrapping matplotlib

In [4]:


# Matplotlib
import matplotlib.pyplot as plt
import plotly.tools as tls

## Generating the data..
x =  np.linspace(-2*np.pi, 2*np.pi, 1000)


fig = plt.figure()                     # Initiate a figure

a1 = fig.add_subplot(221)              # Figure has 1 row 1 column ; choose the first subplot
a1.plot(x,np.sin(x))                   # Plot x and y in the chosen subplot
a1.grid()

a2 = fig.add_subplot(224)
a2.plot(x,np.cos(x))
a2.grid()

In [5]:
# Now we make the matplotlib figure interactive using plotly

plotly_fig = tls.mpl_to_plotly(fig)

plotly.offline.iplot(plotly_fig)

## Plotting columns of a DataFrame

In [1]:
def PlotData(df):
    pdata = [{'x':df.index,'y':df[col],'name':col} for col in df.columns]
    return(pdata)

In [6]:
Prices = pd.read_pickle('ETF_Prices.pkl')
Returns = Prices.pct_change()            # Converts prices to returns


In [14]:
# Plotting price levels
pdata = PlotData(Prices)
layout = go.Layout(title = 'ETF Prices',yaxis=dict(tickformat='.1f',title='Price'))

pFig = go.Figure(pdata,layout)   # creates a Figure object
plotly.offline.iplot(pFig)

In [15]:
# Plotting returns
fig = go.Figure(PlotData(Returns),go.Layout(title='ETF Returns',
                                            yaxis=dict(tickformat='.1%',title='Return')))
fig.show()

## Elaborate subplots

In [19]:
# Rolling Correlations
RCorr = Returns.rolling(12).corr(pairwise=True) # Really nice syntax!

nr = 2 ; nc = 2
fig = make_subplots(rows=nr, cols=nc, start_cell="top-left",
                   subplot_titles=list(RCorr.columns))


In [24]:
# Visualization with subplots. Quite finicky.

for j in range(len(RCorr.columns)):
    ticker = RCorr.columns[j]
    Rows = [(Date,ticker) for Date in RCorr.index.levels[0]]

    df1 = RCorr.loc[Rows,:]
    df1.index = df1.index.levels[0]        # Only Date needed from multi-level index
    df1.drop(columns=ticker,inplace=True)  # All values are 1
    
    fr,fc = np.unravel_index(j,(nr,nc),order='F')  # Fortran style (column major order)
    
    plines = [go.Scatter(x=df1.index,y=df1[col],name=col) for col in df1.columns]
    _=[fig.append_trace(pl,row=int(fr+1),col=int(fc+1)) for pl in plines]
    
                            # only append_trace seems to do the trick with subplots
                            # np.int64 for row and col cause errors, so have to use int
        
fig.show()

## plotly express

Plotly express was inspired by ggplot and Seaborn. It is a wrapper around lower-level classes and methods. It works best if the data are in a long form DataFrame. 

See its [tutorial](https://plot.ly/python/plotly-express/) for details.

In [23]:
import plotly.express as px

RC2 = RCorr.reset_index().rename(columns={'level_1':'ETF'})
RC2 = pd.melt(RC2,id_vars=['Date','ETF'],var_name='With',value_name='Corr')

px.line(RC2,x='Date',y='Corr',color='With',facet_col='ETF')   # Remarkably elegant
